In [9]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd
import re

In [2]:
LIST_PATH = 'https://in-art.ru/art-mir/khudozhniki/'
PREFIX = 'https://in-art.ru'

In [3]:
def get_artist_list(path):    
    page = requests.get(path).content
    soup = bs(page, 'lxml')
    for script in soup.find_all('script'):
        if script.text.find('artistsData') != -1:
            data = script.text
            data = data.replace('null', '"null"')
            data = data[data.find('{'):data.find(r'}};') + 2]
            parsed_artists = json.loads(data, encoding='utf8')['ITEMS']
            break
    for artist in parsed_artists:
        del artist['PICTURES']

    return parsed_artists

In [15]:
def parse_exhibitions(exh):
    res = []
    for elem in exh:
        now_dict = dict()
        if len(elem.strip()) == 4:
            year = elem.strip()
        elif len(elem.strip()) > 10:
            now_dict['name'] = elem[elem.find('«') + 1:elem.find('»')]
            rest = elem[elem.find('»') + 2:].replace('.', ',').split(', ')
            now_dict['museum'] = rest[0].strip()
            now_dict['city'] = rest[1].strip()
            now_dict['year'] = year
            res.append(now_dict)
    return res

In [41]:
def get_artist_data(url):
    page = requests.get(url).content
    soup = bs(page, 'lxml')
    tech = soup.find('div', text=re.compile(r'\s*Техника\s*'))
    if tech is not None:
        tech = tech.parent.find_all('div')[1].find('div').text.strip()
    personal_exh = soup.find(text=re.compile(r'\s*Персональные выставки\s*')).parent.parent.find_all('div')[2].find_all('div')[1].text
    personal_exh = personal_exh.split('\r\n\t')
    personal_exh = parse_exhibitions(personal_exh)
    collective_exh = soup.find(text=re.compile(r'\s*Групповые выставки\s*')).parent.parent.find_all('div')[3].text
    collective_exh = collective_exh.split('\r\n')
    collective_exh = parse_exhibitions(collective_exh)
    return tech, personal_exh, collective_exh

In [42]:
artists_list = get_artist_list(LIST_PATH)

In [43]:
for i in range(2, 10):
    artist = artists_list[i]
    print(artist['NAME'])
    print(get_artist_data(PREFIX + artist['DETAIL_PAGE_URL']), '\n\n')

Megasoma Mars


AttributeError: 'NoneType' object has no attribute 'text'